In [ ]:
import json
import pandas as pd
import numpy as np
from targeted_llm_manipulation.backend.hf_backend import HFBackend
from targeted_llm_manipulation.utils.utils import find_freest_gpus
from pathlib import Path
from targeted_llm_manipulation.data_root import PROJECT_DATA, BENCHMARK_DATA, TRAJ_PATH
from targeted_llm_manipulation.root import PICKLE_SAVE_PATH
from targeted_llm_manipulation.stats.preferences_per_iteration import load_trajs_from_path
import pprint
from targeted_llm_manipulation.utils.utils import load_jsonl
%load_ext autoreload
%autoreload 2


In [ ]:
traj_path = TRAJ_PATH / "weak-therapist1t-env-09_20_121602" / "0"
turns_df, _ = load_trajs_from_path(traj_path)


## Scratch

**Dataset** 

In [ ]:
dataset_filename = "sycophancy/answer.jsonl"
dataset = load_jsonl(BENCHMARK_DATA / dataset_filename)

**Single Inference**

In [ ]:
# Step 1: Map Dataset Prompts to Expected Format
def map_prompt_to_messages(prompt_entry):
    messages = []
    for msg in prompt_entry:
        role = "user" if msg["type"] == "human" else msg["type"]
        messages.append({"role": role, "content": msg["content"]})
    return messages

In [ ]:
messages = [map_prompt_to_messages(entry["prompt"]) for entry in dataset]

In [ ]:
messages[0]

In [ ]:
backend_config = {
    "model_name": "meta-llama/Meta-Llama-3-8B-Instruct",
    "model_id": None,
    "lora_path": None,
}

In [ ]:
device = find_freest_gpus(1)

In [ ]:
backend = HFBackend(device = device[0], **backend_config)

In [ ]:
responses = backend.get_response_vec(
    messages_in=messages[0:3],
    temperature=1,
    max_tokens=1024,
    role=None
)

In [ ]:
responses

In [ ]:
messages[1]

## Using the Dataset Trajectory Generator

**Multi-process Inference**

In [ ]:
from targeted_llm_manipulation.trajectory_generator.dataset_trajectory_generator import DatasetTrajectoryGenerator

In [ ]:
backend_config = {
    "model_name": "meta-llama/Meta-Llama-3-8B-Instruct",
    "model_id": None,
    "lora_path": None,
}
dataset_filename = "sycophancy/answer_small.jsonl"
run_name = "sycophancy_eval"

In [ ]:
generator = DatasetTrajectoryGenerator(
    run_name=run_name,
    dataset_filename=dataset_filename,
    backend_config=backend_config,
    batch_size=10,
    devices=find_freest_gpus(1)
)

In [ ]:
traj_iter_dir = Path(generator.traj_dir) / f"{0}"

responses = generator._multiprocess_generate_trajectories(traj_iter_dir)

In [ ]:
1500/60

**Inspect df**

In [ ]:
from targeted_llm_manipulation.utils.utils import load_pickle

In [ ]:
df = load_pickle(PICKLE_SAVE_PATH / "sycophancy_eval-09-24_20-35.pkl")


In [ ]:
import pprint

In [ ]:
idx = 19
print("The correct answer is: ", df['base'][idx]['correct_answer'])
print("The history is:")
pprint.pprint(df['history'][idx][1])
print("The sycophancy eval is: ", df['sycophancy_eval'][idx])